In [11]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from random import seed
from sklearn.cross_validation import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import cohen_kappa_score
from scipy.special import expit
from scipy.special import logit
from sklearn.cross_validation import train_test_split
%matplotlib inline

In [12]:
# Make Predictions Reproducible
seed(314)

In [13]:
#Turn off SettingWithCopyWarning()
pd.options.mode.chained_assignment = None

In [14]:
!pwd

/Users/Kyle1/Dropbox/GS/Research/Sports/Baseball/Relief-Fatigue/Code


In [38]:
df = pd.read_csv('../Data/pitch_swing.csv', index_col=0)
inds = np.random.choice(df.shape[0], df.shape[0], replace=False)
df = df.iloc[inds].copy()

df_all = pd.read_csv('../Data/AllStandardPitches.csv', index_col=0)
inds_all = np.random.choice(df_all.shape[0], df_all.shape[0], replace=False)
df_all = df_all.iloc[inds_all].copy()

In [19]:
list(df.pitch_type.unique())

['FF', 'CU', 'FT', 'SL', 'FS', 'CH', 'SI', 'FC']

In [40]:
list(df_all.pitch_type.unique())

['SL', 'FF', 'FT', 'CU', 'FS', 'SI', 'CH', 'FC']

In [21]:
df.columns

Index(['gameday_link', 'batter', 'pitcher', 'num', 'event', 'inning_side',
       'inning', 'batter_name', 'pitcher_name', 'date', 'o', 'p_throws', 'des',
       'type', 'x', 'y', 'start_speed', 'end_speed', 'sz_top', 'sz_bot',
       'pfx_x', 'pfx_z', 'px', 'pz', 'x0', 'y0', 'z0', 'vx0', 'vy0', 'vz0',
       'ax', 'ay', 'az', 'break_y', 'break_angle', 'break_length',
       'pitch_type', 'spin_dir', 'spin_rate', 'count', 'zone', 'sv_id',
       'reg_season', 'post_season', 'year', 'b', 's', 'avg_fast_speed',
       'swinging', 'whiff', 'diff_speed'],
      dtype='object')

# Get best leaf for each pitch_type

**Also return table of results**

In [22]:
#k-fold cross validation
def get_best_leaf(df,pitch_list,leaf_list,covariates,folds = 10):

    fits={}
    imps = {}

    for pitch in pitch_list:
        df_p = df[df.pitch_type==pitch]
        fits[pitch] ={}
        Y = df_p.whiff
        X = df_p[covariates]
        avg_scores = []; avg_kappas = [];
        for leaf in leaf_list:
            scores = []; kappas = []; 
            clf = RandomForestClassifier(min_samples_leaf=leaf)

            for train, test in KFold(Y.shape[0], n_folds=folds):
                #Fit model
                clf.fit(X.iloc[train], Y.iloc[train])

                #Judge the model
                scores.append(clf.score(X.iloc[test], Y.iloc[test]))
                preds = clf.predict(X.iloc[test])
                kappas.append(cohen_kappa_score(preds, Y.iloc[test]))

            #print('\nNumber of Leaves: {}'.format(leaf))
            #print('Avg Score: {0:.3f}'.format(np.mean(scores)))
            #print('Sd Score: {0:.2f}'.format(np.std(scores)))
            #print('Avg Kappa: {0:.3f}'.format(np.mean(kappas)))
            avg_scores.append(round(np.mean(scores),3))
            avg_kappas.append(round(np.mean(kappas),3))

        best_k = np.argmax(avg_kappas)
        best_s = np.argmax(avg_scores)
        print('\nPitch Type: {}'.format(pitch))
        print("Percent Contact: ", round(1-Y.mean(),3))
        print('Leaves/Score/Kappa for best:')
        print('Score: ',leaf_list[best_s],"/",avg_scores[best_s],"/",avg_kappas[best_s])
        print('Kappa: ',leaf_list[best_k],"/",avg_scores[best_k],"/",avg_kappas[best_k])

        fits[pitch]['perc_whiff'] = round(1-Y.mean(),3)
        fits[pitch]['score'] = avg_scores[best_s]
        fits[pitch]['num_pitches'] = int(len(Y))
        fits[pitch]['leaf'] = int(leaf_list[best_s])
        fits[pitch]['kappa'] = avg_kappas[best_s]

    return(fits)



In [23]:
leafs = [25,50,100,200,500];
pitch_types = list(df.pitch_type.unique())
covars = ['start_speed','end_speed','pfx_x','vx0','vy0','vz0','pfx_z','ax','ay','az','break_y',
         'break_angle','spin_dir','spin_rate']
best = get_best_leaf(df,pitch_types,leafs,covars)


Pitch Type: FF
Percent Contact:  0.759
Leaves/Score/Kappa for best:
Score:  50 / 0.76 / 0.036
Kappa:  25 / 0.759 / 0.053

Pitch Type: CU
Percent Contact:  0.533
Leaves/Score/Kappa for best:
Score:  25 / 0.707 / 0.407
Kappa:  25 / 0.707 / 0.407

Pitch Type: FT
Percent Contact:  0.799
Leaves/Score/Kappa for best:
Score:  25 / 0.802 / 0.052
Kappa:  25 / 0.802 / 0.052

Pitch Type: SL
Percent Contact:  0.545
Leaves/Score/Kappa for best:
Score:  50 / 0.684 / 0.353
Kappa:  50 / 0.684 / 0.353

Pitch Type: FS
Percent Contact:  0.553
Leaves/Score/Kappa for best:
Score:  25 / 0.663 / 0.307
Kappa:  25 / 0.663 / 0.307

Pitch Type: CH
Percent Contact:  0.536
Leaves/Score/Kappa for best:
Score:  25 / 0.628 / 0.246
Kappa:  25 / 0.628 / 0.246

Pitch Type: SI
Percent Contact:  0.798
Leaves/Score/Kappa for best:
Score:  25 / 0.802 / 0.058
Kappa:  25 / 0.802 / 0.058

Pitch Type: FC
Percent Contact:  0.695
Leaves/Score/Kappa for best:
Score:  25 / 0.715 / 0.146
Kappa:  25 / 0.715 / 0.146


# Get Predicted Means Table

In [24]:
def get_pred_means_table(df,covariates,leaf, subset = None):
    df_p_train, df_p_test = train_test_split(df, test_size=0.5, random_state=42)

    #Test and train X datasets
    Y_train = df_p_train.whiff
    Y_test = df_p_test.whiff
    X_train = df_p_train[covariates]

    X_test = df_p_test[covariates]


    X_train = pd.concat([pd.get_dummies(X_train[col]).ix[:, :-1] 
                   if X_train[col].dtype == object or hasattr(X_train[col], 'cat')
                   else X_train[col]
                   for col in X_train.columns], axis=1)

    X_test = pd.concat([pd.get_dummies(X_test[col]).ix[:, :-1] 
                   if X_test[col].dtype == object or hasattr(X_test[col], 'cat')
                   else X_test[col]
                   for col in X_test.columns], axis=1)

    #Fit the Random Forest with the best leaf
    clf = RandomForestClassifier(min_samples_leaf=leaf)
    clf.fit(X_train,Y_train)

    print(1-Y_test.mean())
    print(clf.score(X_test,Y_test))
    #Predict probabilities
    pred_probs = clf.predict_proba(X_test)[:,1] #predicted probability of strike

    #Take the logit
    pred_stuff = logit(pred_probs)

    pred_mat = df_p_test[["pitcher_name"]].copy()
    pred_mat['pred_stuff'] = pred_stuff
    
    #Get rid of people with predicted probability 0
    pred_mat['pred_stuff'] = pred_mat['pred_stuff'][pred_mat['pred_stuff']>-math.inf]
    
    #mean/sd for standardization
    overall_mean = pred_mat.pred_stuff.mean()
    overall_sd = pred_mat.pred_stuff.std()
    
    #Standardize
    pred_mat['pred_z'] = (pred_mat['pred_stuff'] - overall_mean)/overall_sd
    
    #group by pitcher name
    #average the stuff
    group_pred = pred_mat.groupby('pitcher_name')

    mapped_funs = {'pred_z': {'count','mean'}}
    pred_means = group_pred.agg(mapped_funs)

    #Only show me players with more than 100 of that pitch
    if(subset is not None):
        pred_means = pred_means[pred_means[('pred_z','count')]>subset]
    
    pred_means = pred_means.sort_values(('pred_z','mean'),ascending = False)

    return(pred_means)

# Train Model based on best leaf, attach to other dset

** Need to train model on 2012/2016 swings, but attach to 2013-2015 all pitches**

In [25]:
def add_stuff(train_dset, pred_dset, best_fits, covariates):
    
    #Train all the pitches, save the clfs
    all_pitches = train_dset.pitch_type.unique()
    if(len(all_pitches)!=len(best_fits)):
        print('Error: Num pitch types in fits dictionary not same as num pitch types in dset')
        return
    
    clf_dict = {}
    for i in range(len(all_pitches)):
        pitch = all_pitches[i]
        
        if(pitch not in best_fits.keys()):
            print(pitch+' not in fits dictionary')
            return
        
        train_df = train_dset[train_dset.pitch_type==pitch]
        X_train = train_df[covariates]

        X_train = pd.concat([pd.get_dummies(X_train[col]).ix[:, :-1] 
                       if X_train[col].dtype == object or hasattr(X_train[col], 'cat')
                       else X_train[col]
                       for col in X_train.columns], axis=1)
        
        Y_train = train_df.whiff
        
        leaf = best_fits[pitch]['leaf']
        clf =  RandomForestClassifier(min_samples_leaf=leaf)
        clf.fit(X_train,Y_train)
        
        ##Now, predict on new data
        pred_df = pred_dset[pred_dset.pitch_type==pitch].copy()
        
        X_pred = pred_df[covariates]
        X_pred = pd.concat([pd.get_dummies(X_pred[col]).ix[:, :-1] 
                       if X_pred[col].dtype == object or hasattr(X_pred[col], 'cat')
                       else X_pred[col]
                       for col in X_pred.columns], axis=1)
        
        #Predict probabilities
        pred_probs = clf.predict_proba(X_pred)[:,1] #predicted probability of strike

        #Take the logit
        pred_stuff = logit(pred_probs)
        
        overall_mean = pred_stuff.mean()
        overall_sd = pred_stuff.std()
        
        pred_z = (pred_stuff - overall_mean)/overall_sd
        
        pred_df['z_stuff'] = pred_z
        
        if(i==0):
            out_mat = pred_df.copy()
        else:
            out_mat = out_mat.append(pred_df)
        
    
    if(out_mat.shape[0]!= pred_dset.shape[0]):
        print("You got more than zero problems big fella")
        
    return(out_mat)

    

In [41]:
final_train = df[(df.year==2016) | (df.year==2012)]
final_pred = df_all[(df_all.year>=2013)&(df_all.year<=2015)]

In [42]:
jags_dset = add_stuff(final_train, final_pred,best,covars,)

In [43]:
jags_group = jags_dset.groupby('pitcher_name')

mapped_funs = {'z_stuff': {'count','mean'}}
z_stuff = jags_group.agg(mapped_funs)
z_stuff.sort_values(('z_stuff','mean'),ascending=False)

z_stuff      
                         mean count
pitcher_name                       
Aroldis Chapman      0.944124  3072
Zachary Britton      0.910841  1795
Jacob McGee          0.884618  2779
Matt Daley           0.836467   128
Scott Rice           0.816174   959
Austin Adams         0.775650   351
Craig Kimbrel        0.766897  3126
Jeurys Familia       0.710812  2444
Andrew Bailey        0.650001   451
Michael Kohn         0.636964  1143
Michael Foltynewicz  0.581923   151
Jeff Beliveau        0.547635   141
Jose Dominguez       0.543010   115
Enny Romero          0.538539   518
Mitchell Boggs       0.538067   156
Carter Capps         0.533759  1609
Blake Treinen        0.532910   918
Ernesto Frieri       0.518603  2292
Darin Downs          0.506671   914
Kyuji Fujikawa       0.506308   251
Cameron Bedrosian    0.501433   834
Brayan Villarreal    0.491629   111
Rafael Martin        0.470097   171
Rhiner Cruz          0.463397   216
Sean Marshall        0.450887   102
Joshua Fields        0.423389  2408
Kenneth Giles        0.419996  1847
Ross Ohlendorf       0.417839   408
Kevin Jepsen         0.409994  2675
Bryan Morris         0.406608  2642
...                       ...   ...
Ryan Mattheus       -0.418841  1190
Drew VerHagen       -0.420830   287
Brian Wilson        -0.421401  1010
Sam LeCure          -0.439673  1877
Pat Neshek          -0.446268  2463
Miguel Socolovich   -0.447391   487
Chris Resop         -0.451869   129
Matt Reynolds       -0.457591   431
Burke Badenhop      -0.460788  2902
Colt Hynes          -0.466692   231
Eury De La Rosa     -0.468829   403
Jean Machi          -0.487821  2318
Jeff Francis        -0.488305   264
Paul Clemens        -0.489278   222
Thad Weber          -0.504242   213
Zeke Spruill        -0.511286   126
Jacob Brigham       -0.518479   131
Daniel Otero        -0.523171  2225
Sugar Ray Marimon   -0.537277   258
Michael Tonkin      -0.541402   610
Collin Balester     -0.548616   180
Keith Hessler       -0.548902   148
Cory Burns          -0.550800   103
LaTroy Hawkins      -0.586628  2074
Eric Surkamp        -0.657880   115
Kevin Gregg         -0.659520  1156
Kenneth Roberts     -0.691852   137
Cesar Jimenez       -0.696376   554
Mike Adams          -0.719703   272
Blake Parker        -0.748604   984

[475 rows x 2 columns]

In [44]:
jags_dset.shape

(541991, 52)

In [46]:
jags_dset.to_csv('../Data/all_pitches_pred.csv')